In [13]:
import pandas as pd
import numpy as np

file = 'Base anonimizada - Eric - PUC-SP.xlsx'

data = pd.read_excel(file, skiprows=2)

dates = pd.read_excel(file)
dates = (
    dates.drop(columns=[col for col in dates.columns if 'Unnamed:' in col])
    .iloc[0]
    .to_dict()
)

colunas_por_aula = ['Pre-Class', 'P', 'Hw', 'CP', 'Bh']
id_columns = ['Nome Planilha Feedback', 'Sala', 'Num', 'NOME COMPLETO']


def is_class_column(col: str) -> bool:
    """Verifica se a coluna faz parte do grupo de notas por aula."""
    return col.split('.')[0] in colunas_por_aula


def split_colum_values(col: str):
    """Divide a coluna entre o nome base e o número da aula (ou 0 se não tiver)."""
    try:
        coluna, aula = col.split('.')
        return is_class_column(coluna), coluna, int(aula)
    except:
        return is_class_column(col), col, 0


def get_class_date(aula_idx: int):
    """Obtém a data da aula a partir do dicionário de datas."""
    aula_key = f'Aula {aula_idx + 1}' if aula_idx > 0 else 'Aula 1'
    return dates.get(aula_key, None)


class_columns = [col for col in data.columns if is_class_column(col)]

all_classes = []

for col in class_columns:
    needed, col_name, aula_idx = split_colum_values(col)
    if not needed:
        continue
    group_cols = [f'{c}.{aula_idx}' if f'{c}.{aula_idx}' in data.columns else c for c in colunas_por_aula]
    aula_df = data[id_columns + [c for c in group_cols if c in data.columns]].copy()
    aula_df['Aula'] = aula_idx + 1
    aula_df['Data'] = get_class_date(aula_idx)
    aula_df.rename(columns={c: c.split('.')[0] for c in group_cols if '.' in c}, inplace=True)
    all_classes.append(aula_df)

compiled = pd.concat(all_classes, ignore_index=True)

compiled.sort_values(by=['Nome Planilha Feedback', 'Aula'], inplace=True)



In [14]:
compiled

,Nome Planilha Feedback,Sala,Num,NOME COMPLETO,Pre-Class,P,Hw,CP,Bh,Aula,Data
108,Four Corners A1 - 1S2025 - Campinas - Feedback,Four Corners A1,1,Estudante 1 - Campinas Four Corners A1,NaN,P,NaN,:-D,NaN,1,8-fev.-2025
109,Four Corners A1 - 1S2025 - Campinas - Feedback,Four Corners A1,2,Estudante 2 - Campinas Four Corners A1,NaN,P,NaN,:-D,NaN,1,8-fev.-2025
110,Four Corners A1 - 1S2025 - Campinas - Feedback,Four Corners A1,3,Estudante 3 - Campinas Four Corners A1,NaN,P,NaN,:-D,NaN,1,8-fev.-2025
111,Four Corners A1 - 1S2025 - Campinas - Feedback,Four Corners A1,4,Estudante 4 - Campinas Four Corners A1,NaN,P,NaN,:-D,NaN,1,8-fev.-2025
112,Four Corners A1 - 1S2025 - Campinas - Feedback,Four Corners A1,5,Estudante 5 - Campinas Four Corners A1,NaN,P,NaN,:-D,NaN,1,8-fev.-2025
...,...,...,...,...,...,...,...,...,...,...,...
140152,Four Corners J3 - 1S2025 - Unidade Virtual (au...,Four Corners J3,14,Estudante 14 - Unidade Virtual (aulas aos sába...,NaN,11,NaN,:-D,NaN,15,None
140153,Four Corners J3 - 1S2025 - Unidade Virtual (au...,Four Corners J3,15,Estudante 15 - Unidade Virtual (aulas aos sába...,NaN,12,NaN,:-D,NaN,15,None
140154,Four Corners J3 - 1S2025 - Unidade Virtual (au...,Four Corners J3,16,Estudante 16 - Unidade Virtual (aulas aos sába...,NaN,13,NaN,:-D,NaN,15,None
140155,Four Corners J3 - 1S2025 - Unidade Virtual (au...,Four Corners J3,17,Estudante 17 - Unidade Virtual (aulas aos sába...,NaN,13,NaN,:-D,NaN,15,None


In [15]:
# Seleciona apenas as colunas relevantes
data_unidades = data[['NOME COMPLETO', 'Nome Planilha Feedback', 'Sala']].copy()

# Extrai o nome da unidade com segurança
def extrair_unidade(texto):
    partes = str(texto).split(' - ')
    return partes[2] if len(partes) > 2 else None

data_unidades['unidade'] = data_unidades['Nome Planilha Feedback'].apply(extrair_unidade)

# Remove duplicados (cada aluno aparece só uma vez)
data_alunos = data_unidades[['NOME COMPLETO', 'unidade', 'Sala']].drop_duplicates()



In [16]:
# Quantos alunos há por unidade
print("📊 Alunos por unidade:")
data_alunos.groupby('unidade')['NOME COMPLETO'].count()

📊 Alunos por unidade:


unidade
Campinas                                241
Capão Redondo                           108
Diadema                                 141
Jabaquara                               145
Mauá                                     53
Rio de Janeiro                          147
Tatuapé                                 157
Unidade Virtual (aulas aos domingos)    210
Unidade Virtual (aulas aos sábados)     445
Vila Sônia (manhã)                      195
Vila Sônia (tarde)                      135
Name: NOME COMPLETO, dtype: int64

In [17]:

# Quantos alunos há por sala e unidade
print("📊 Alunos por sala e unidade:")
data_alunos.groupby(['Sala', 'unidade'])['NOME COMPLETO'].count()

📊 Alunos por sala e unidade:


Sala             unidade                            
Four Corners A1  Campinas                               16
                 Capão Redondo                          19
                 Diadema                                21
                 Jabaquara                              22
                 Mauá                                   13
                                                        ..
Four Corners J1  Unidade Virtual (aulas aos sábados)    18
                 Vila Sônia (manhã)                     14
Four Corners J2  Campinas                               10
                 Unidade Virtual (aulas aos sábados)    18
Four Corners J3  Unidade Virtual (aulas aos sábados)    18
Name: NOME COMPLETO, Length: 125, dtype: int64

In [28]:
import pandas as pd
import numpy as np

file = 'Base anonimizada - Eric - PUC-SP.xlsx'

# --- 1. Ler os dados principais e as datas ---
data = pd.read_excel(file, skiprows=2)

dates = pd.read_excel(file)
dates = (
    dates.drop(columns=[col for col in dates.columns if 'Unnamed:' in col])
    .iloc[0]
    .to_dict()
)

# --- 2. Definir colunas por aula e funções auxiliares ---
colunas_por_aula = ['Pre-Class', 'P', 'Hw', 'CP', 'Bh']
id_columns = ['Nome Planilha Feedback', 'Sala', 'Num', 'NOME COMPLETO']

def is_class_column(col: str) -> bool:
    return col.split('.')[0] in colunas_por_aula

def split_colum_values(col: str):
    try:
        coluna, aula = col.split('.')
        return is_class_column(coluna), coluna, int(aula)
    except:
        return is_class_column(col), col, 0

def get_class_date(aula_idx: int):
    aula_key = f'Aula {aula_idx + 1}' if aula_idx > 0 else 'Aula 1'
    return dates.get(aula_key, None)

# --- 3. Reorganizar dados por aula ---
class_columns = [col for col in data.columns if is_class_column(col)]
all_classes = []

for col in class_columns:
    needed, col_name, aula_idx = split_colum_values(col)
    if not needed:
        continue
    group_cols = [f'{c}.{aula_idx}' if f'{c}.{aula_idx}' in data.columns else c for c in colunas_por_aula]
    aula_df = data[id_columns + [c for c in group_cols if c in data.columns]].copy()
    aula_df['Aula'] = aula_idx + 1
    aula_df['Data'] = get_class_date(aula_idx)
    aula_df.rename(columns={c: c.split('.')[0] for c in group_cols if '.' in c}, inplace=True)
    all_classes.append(aula_df)

compiled = pd.concat(all_classes, ignore_index=True)

# --- 4. Adicionar colunas de Aluno e Unidade ---
def extrair_unidade(texto):
    partes = str(texto).split(' - ')
    return partes[2] if len(partes) > 2 else None

compiled['Aluno'] = compiled['NOME COMPLETO'].apply(lambda x: str(x).split(' - ')[0])
compiled['Unidade'] = compiled['Nome Planilha Feedback'].apply(extrair_unidade)

# --- 5. Montar base final limpa ---
base_final = compiled[[
    'Aluno', 'Unidade', 'Sala', 'Aula', 'Data',
    'Pre-Class', 'P', 'Hw', 'CP', 'Bh'
]].drop_duplicates()

# --- 6. (Opcional) Ordenar e salvar ---
base_final.sort_values(by=['Unidade', 'Sala', 'Aluno', 'Aula'], inplace=True)

# Salvar em Excel, se quiser:
# base_final.to_excel('base_final.xlsx', index=False)

# Exibir primeiras linhas
base_final

,Aluno,Unidade,Sala,Aula,Data,Pre-Class,P,Hw,CP,Bh
108,Estudante 1,Campinas,Four Corners A1,1,8-fev.-2025,NaN,P,NaN,:-D,NaN
9993,Estudante 1,Campinas,Four Corners A1,2,15-fev.-2025,√,P,√,:-D,NaN
19878,Estudante 1,Campinas,Four Corners A1,3,22-fev.-2025,√,P,√,:-D,NaN
29763,Estudante 1,Campinas,Four Corners A1,4,8-mar.-2025,√,P,√,:-D,NaN
39648,Estudante 1,Campinas,Four Corners A1,5,15-mar.-2025,√,P,√,:-D,NaN
...,...,...,...,...,...,...,...,...,...,...
100024,Estudante 6,Vila Sônia (tarde),Four Corners I1,11,24-mai.-2025,√,P,√,:-D,NaN
109909,Estudante 6,Vila Sônia (tarde),Four Corners I1,12,31-mai.-2025,√,P,√,:-D,NaN
119794,Estudante 6,Vila Sônia (tarde),Four Corners I1,13,7-jun.-2025,√,P,NaN,:-D,NaN
129679,Estudante 6,Vila Sônia (tarde),Four Corners I1,14,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
base_final.Data.unique()

array(['8-fev.-2025', '15-fev.-2025', '22-fev.-2025', '8-mar.-2025',
       '15-mar.-2025', '29-mar.-2025', '5-abr.-2025', '12-abr.-2025',
       '26-abr.-2025', '17-mai.-2025', '24-mai.-2025', '31-mai.-2025',
       '7-jun.-2025', nan, None], dtype=object)

In [52]:
def parse_portuguese_dates(dates):
    if isinstance(dates, pd.Series):
        dates = dates.astype(str).replace(['nan', 'None'], np.nan)
    elif not isinstance(dates, list):
        raise TypeError("O argumento deve ser uma lista ou pd.Series.")
    
    meses = {
        'jan.': 'Jan', 'fev.': 'Feb', 'mar.': 'Mar', 'abr.': 'Apr',
        'mai.': 'May', 'jun.': 'Jun', 'jul.': 'Jul', 'ago.': 'Aug',
        'set.': 'Sep', 'out.': 'Oct', 'nov.': 'Nov', 'dez.': 'Dec'
    }

    datas_en = []
    for d in dates:
        if isinstance(d, str):
            for pt, en in meses.items():
                d = d.replace(pt, en)
        datas_en.append(d)

    return pd.to_datetime(datas_en, format='%d-%b-%Y', errors='coerce')

base_final['Data'] = parse_portuguese_dates(base_final.Data.to_list())

In [53]:
base_final.to_csv('eric-dados-compilados.csv', index=None)

In [54]:
base_final.columns

Index(['Aluno', 'Unidade', 'Sala', 'Aula', 'Data',
       'Fez a atividade antes da aula', 'Presença/Ausencia',
       'Fez lição de casa', 'Participação', 'Comportamento'],
      dtype='object')

In [55]:
colunas_corno = [ 'Pre-Class', 'P', 'Hw',
       'CP', 'Bh']

In [56]:


metadata = pd.read_excel(file, sheet_name=['Metadados'])['Metadados']



In [57]:
metadata.columns = ['detalhes', 'coluna', 'nome_novo']

In [58]:
metadata[metadata['coluna'].isin(colunas_corno)][['coluna', 'nome_novo']].to_dict(orient='records')

[{'coluna': 'Pre-Class', 'nome_novo': 'Fez a atividade antes da aula'},
 {'coluna': 'P', 'nome_novo': 'Presença/Ausencia'},
 {'coluna': 'Hw', 'nome_novo': 'Fez lição de casa'},
 {'coluna': 'CP', 'nome_novo': 'Participação'},
 {'coluna': 'Bh', 'nome_novo': 'Comportamento'},
 {'coluna': 'P', 'nome_novo': nan}]

In [59]:
colunas_novas = [{'coluna': 'Pre-Class', 'nome_novo': 'Fez a atividade antes da aula'},
 {'coluna': 'P', 'nome_novo': 'Presença/Ausencia'},
 {'coluna': 'Hw', 'nome_novo': 'Fez lição de casa'},
 {'coluna': 'CP', 'nome_novo': 'Participação'},
 {'coluna': 'Bh', 'nome_novo': 'Comportamento'}]

In [60]:
fds = pd.DataFrame(colunas_novas)

In [63]:
base_final.to_csv('eric-dados-compilados-com-nomes.csv', index=None)    


In [64]:
base_final

,Aluno,Unidade,Sala,Aula,Data,Fez a atividade antes da aula,Presença/Ausencia,Fez lição de casa,Participação,Comportamento
108,Estudante 1,Campinas,Four Corners A1,1,2025-02-08,NaN,P,NaN,:-D,NaN
9993,Estudante 1,Campinas,Four Corners A1,2,2025-02-15,√,P,√,:-D,NaN
19878,Estudante 1,Campinas,Four Corners A1,3,2025-02-22,√,P,√,:-D,NaN
29763,Estudante 1,Campinas,Four Corners A1,4,2025-03-08,√,P,√,:-D,NaN
39648,Estudante 1,Campinas,Four Corners A1,5,2025-03-15,√,P,√,:-D,NaN
...,...,...,...,...,...,...,...,...,...,...
100024,Estudante 6,Vila Sônia (tarde),Four Corners I1,11,2025-05-24,√,P,√,:-D,NaN
109909,Estudante 6,Vila Sônia (tarde),Four Corners I1,12,2025-05-31,√,P,√,:-D,NaN
119794,Estudante 6,Vila Sônia (tarde),Four Corners I1,13,2025-06-07,√,P,NaN,:-D,NaN
129679,Estudante 6,Vila Sônia (tarde),Four Corners I1,14,NaT,NaN,NaN,NaN,NaN,NaN


In [65]:
base_final.dropna(subset=['Data'], inplace=True)

In [70]:
base_final.columns

Index(['Aluno', 'Unidade', 'Sala', 'Aula', 'Data',
       'Fez a atividade antes da aula', 'Presença/Ausencia',
       'Fez lição de casa', 'Participação', 'Comportamento'],
      dtype='object')

In [71]:
colunas_corno = ['Fez a atividade antes da aula', 'Presença/Ausencia', 'Fez lição de casa', 'Participação', 'Comportamento']

In [73]:
colunas_corno

['Fez a atividade antes da aula',
 'Presença/Ausencia',
 'Fez lição de casa',
 'Participação',
 'Comportamento']

In [ ]:
for col in colunas_corno:
    print(base_final[col].unique())

[nan '√' 'N' '+/-' '#ERROR!']
['P' 'A' nan '1/2' datetime.datetime(2025, 1, 2, 0, 0) 'p']
[nan '√' 'N' '+/-']
[':-D' nan ':-/' ':-&']
[nan ':-||' ':-(']


In [86]:
base_final[base_final['Fez a atividade antes da aula'] == '#ERROR!']

,Aluno,Unidade,Sala,Aula,Data,Fez a atividade antes da aula,Presença/Ausencia,Fez lição de casa,Participação,Comportamento
11725,Estudante 14,Unidade Virtual (aulas aos domingos),Four Corners D1,2,2025-02-15,#ERROR!,P,√,:-/,NaN


In [87]:
base_final =  base_final.drop(base_final[base_final['Fez a atividade antes da aula'] == '#ERROR!'].index)

In [93]:
base_final[ pd.isna(base_final['Fez a atividade antes da aula'])]

,Aluno,Unidade,Sala,Aula,Data,Fez a atividade antes da aula,Presença/Ausencia,Fez lição de casa,Participação,Comportamento
108,Estudante 1,Campinas,Four Corners A1,1,2025-02-08,NaN,P,NaN,:-D,NaN
117,Estudante 10,Campinas,Four Corners A1,1,2025-02-08,NaN,P,NaN,:-D,NaN
118,Estudante 11,Campinas,Four Corners A1,1,2025-02-08,NaN,P,NaN,:-D,NaN
119,Estudante 12,Campinas,Four Corners A1,1,2025-02-08,NaN,NaN,NaN,NaN,NaN
10004,Estudante 12,Campinas,Four Corners A1,2,2025-02-15,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1170,Estudante 2,Vila Sônia (tarde),Four Corners I1,1,2025-02-08,NaN,P,NaN,:-D,NaN
1171,Estudante 3,Vila Sônia (tarde),Four Corners I1,1,2025-02-08,NaN,P,NaN,:-D,NaN
1172,Estudante 4,Vila Sônia (tarde),Four Corners I1,1,2025-02-08,NaN,A,NaN,NaN,NaN
1173,Estudante 5,Vila Sônia (tarde),Four Corners I1,1,2025-02-08,NaN,P,NaN,:-D,NaN


#


In [95]:
base_final[ (base_final['Presença/Ausencia'] == 'A') & ~(base_final['Participação'].isna())]  

,Aluno,Unidade,Sala,Aula,Data,Fez a atividade antes da aula,Presença/Ausencia,Fez lição de casa,Participação,Comportamento
98967,Estudante 10,Campinas,Four Corners A1,11,2025-05-24,√,A,√,:-D,NaN
108857,Estudante 15,Campinas,Four Corners A1,12,2025-05-31,√,A,√,:-D,NaN
108849,Estudante 7,Campinas,Four Corners A1,12,2025-05-31,√,A,√,:-D,NaN
29789,Estudante 11,Campinas,Four Corners B1,4,2025-03-08,√,A,√,:-D,NaN
29792,Estudante 14,Campinas,Four Corners B1,4,2025-03-08,√,A,√,:-D,NaN
...,...,...,...,...,...,...,...,...,...,...
60419,Estudante 15,Vila Sônia (tarde),Four Corners C2,7,2025-04-05,√,A,√,:-D,NaN
99960,Estudante 16,Vila Sônia (tarde),Four Corners C2,11,2025-05-24,N,A,√,:-D,NaN
70293,Estudante 4,Vila Sônia (tarde),Four Corners C2,8,2025-04-12,√,A,√,:-D,NaN
99952,Estudante 8,Vila Sônia (tarde),Four Corners C2,11,2025-05-24,√,A,N,:-D,NaN


In [96]:
base_final

,Aluno,Unidade,Sala,Aula,Data,Fez a atividade antes da aula,Presença/Ausencia,Fez lição de casa,Participação,Comportamento
108,Estudante 1,Campinas,Four Corners A1,1,2025-02-08,NaN,P,NaN,:-D,NaN
9993,Estudante 1,Campinas,Four Corners A1,2,2025-02-15,√,P,√,:-D,NaN
19878,Estudante 1,Campinas,Four Corners A1,3,2025-02-22,√,P,√,:-D,NaN
29763,Estudante 1,Campinas,Four Corners A1,4,2025-03-08,√,P,√,:-D,NaN
39648,Estudante 1,Campinas,Four Corners A1,5,2025-03-15,√,P,√,:-D,NaN
...,...,...,...,...,...,...,...,...,...,...
80254,Estudante 6,Vila Sônia (tarde),Four Corners I1,9,2025-04-26,+/-,P,√,:-D,NaN
90139,Estudante 6,Vila Sônia (tarde),Four Corners I1,10,2025-05-17,√,A,√,NaN,NaN
100024,Estudante 6,Vila Sônia (tarde),Four Corners I1,11,2025-05-24,√,P,√,:-D,NaN
109909,Estudante 6,Vila Sônia (tarde),Four Corners I1,12,2025-05-31,√,P,√,:-D,NaN


In [97]:
base_final['id'] = base_final['Aluno'] + base_final['Unidade']+ base_final['Sala'] 

In [ ]:
base_final

In [ ]:
def set_presnce_rule(row):
    if row['Presença/Ausencia'] != 'P' and row['Participação'] != ':-D':
        return 'P'


    if pd.isna(row['Presença/Ausencia']):
        return 'A'
    


    return row['Presença/Ausencia']

In [113]:
base_final['Presença/Ausencia1'] = base_final.apply(set_presnce_rule, axis=1)

In [114]:
base_final

,Aluno,Unidade,Sala,Aula,Data,Fez a atividade antes da aula,Presença/Ausencia,Fez lição de casa,Participação,Comportamento,id,Presença/Ausencia1
108,Estudante 1,Campinas,Four Corners A1,1,2025-02-08,NaN,P,NaN,:-D,NaN,Estudante 1CampinasFour Corners A1,P
9993,Estudante 1,Campinas,Four Corners A1,2,2025-02-15,√,P,√,:-D,NaN,Estudante 1CampinasFour Corners A1,P
19878,Estudante 1,Campinas,Four Corners A1,3,2025-02-22,√,P,√,:-D,NaN,Estudante 1CampinasFour Corners A1,P
29763,Estudante 1,Campinas,Four Corners A1,4,2025-03-08,√,P,√,:-D,NaN,Estudante 1CampinasFour Corners A1,P
39648,Estudante 1,Campinas,Four Corners A1,5,2025-03-15,√,P,√,:-D,NaN,Estudante 1CampinasFour Corners A1,P
...,...,...,...,...,...,...,...,...,...,...,...,...
80254,Estudante 6,Vila Sônia (tarde),Four Corners I1,9,2025-04-26,+/-,P,√,:-D,NaN,Estudante 6Vila Sônia (tarde)Four Corners I1,P
90139,Estudante 6,Vila Sônia (tarde),Four Corners I1,10,2025-05-17,√,A,√,NaN,NaN,Estudante 6Vila Sônia (tarde)Four Corners I1,P
100024,Estudante 6,Vila Sônia (tarde),Four Corners I1,11,2025-05-24,√,P,√,:-D,NaN,Estudante 6Vila Sônia (tarde)Four Corners I1,P
109909,Estudante 6,Vila Sônia (tarde),Four Corners I1,12,2025-05-31,√,P,√,:-D,NaN,Estudante 6Vila Sônia (tarde)Four Corners I1,P
